In [1]:
import torch
import torch.nn as nn
from data.pyg_dataToGraph import DataToGraph
import torch.nn.functional as F
from matplotlib import pyplot as plt
from src.Unet import ConditionalDiffusionUNet
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import numpy as np
from src.gaussian_diffusion import GaussianDiffusion

In [2]:
# TODO 加载数据集
dataset = DataToGraph(
    raw_data_path='../data/raw',
    dataset_name='TFF' + '.mat')  # 格式: [(graph,label),...,(graph,label)]

input_dim = dataset[0].x.size(1)
num_classes = dataset.num_classes

# 提取所有的x和y
x0 = []
labels = []

for data in dataset:
    # 提取x (形状为 [num_nodes, input_dim])
    # 但是你提到dataset.x的形状是 [24,50]，这可能是一个图的x特征矩阵
    x0.append(data.x)
    # 提取y（标量标签）
    labels.append(data.y)

# 将列表转换为张量
x0 = torch.stack(x0).unsqueeze(1)  # 形状 [num_samples, 1, 24, 50]

labels = torch.stack(labels)  # 形状 [num_samples]

print(num_classes)
print("X0 shape:", x0.shape)
print("Labels shape:", labels.shape)

7
X0 shape: torch.Size([2368, 1, 24, 50])
Labels shape: torch.Size([2368, 1])


Processing...
Done!


In [3]:
# 将数据传输到GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 初始化模型
model = ConditionalDiffusionUNet(
    num_classes=num_classes,
    time_dim=128,
    label_dim=64
).to(device)
# 准备数据集
dataset = TensorDataset(x0, labels)  # x0: [N,24,50], labels: [N]
dataloader = DataLoader(dataset,
                    batch_size=64,
                    shuffle=True,
                    pin_memory=True)


In [4]:
num_epochs = 1000
lr = 3e-4
grad_clip = 1.0
save_interval = 50  # 每50个epoch保存一次模型

# 初始化扩散模型
diffusion = GaussianDiffusion(num_steps=64,model = model, )

# 初始化优化器和学习率调度器
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

# 将数据转移到GPU
x0 = x0.to(device)
labels = labels.to(device)

best_loss = float('inf')
train_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    with tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}") as pbar:
        for batch_idx, (x_batch, label_batch) in enumerate(pbar):
            # 数据转移到GPU
            x_batch = x_batch.to(device)  # [B, 24, 50]
            label_batch = label_batch.to(device)

            # 随机采样时间步 (重要!)
            B = x_batch.size(0)
            t = torch.randint(0, diffusion.num_steps, (B,), device=device).long()

            # 前向传播计算损失
            losses = diffusion.training_losses(
                model=model,
                x_start=x_batch,
                t=t,
                batch_labels=label_batch
            )
            loss = losses["loss"].mean()

            # 反向传播
            optimizer.zero_grad()
            loss.backward()

            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            # 参数更新
            optimizer.step()

            # 记录损失
            epoch_loss += loss.item()
            pbar.set_postfix({
                'Loss': loss.item(),
                'MSE': losses['mse'].mean().item(),
                'VB': losses['vb'].mean().item()
            })

    # 计算平均epoch损失
    avg_epoch_loss = epoch_loss / len(dataloader)
    train_losses.append(avg_epoch_loss)

    # 更新学习率
    scheduler.step()

    # 保存最佳模型
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        torch.save(model.state_dict(), f'best_unet.pth')
        print(f"New best model saved with loss {best_loss:.4f}")

    # 定期保存检查点
    if (epoch + 1) % save_interval == 0:
        torch.save({
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'loss': avg_epoch_loss,
        }, f'checkpoint_epoch_{epoch + 1}.pth')

    # 绘制损失曲线
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.legend()
    plt.savefig('loss_curve.png')
    plt.close()

print("Training completed!")


TypeError: GaussianDiffusion.__init__() missing 2 required positional arguments: 'model' and 'classifier'